First steps towards a sample of spec/photometric Ias. 

Final goal will be to have saltfits (based on both bts, ampel-spec and ampel-photo z), host dist/mass (LS?) and BTS type for a large sample.

Steps:
* Create a filter query and process candidates. (this notebook)
* Run a bunch of T2s (next).


Q1: How long does it take to process one week of alerts? (and how many do that end up being)
Roughly started at 14.40 Let it run to 15.30. At that time roughly a third of ~15000 chunks were processed. Saved O(200) transients. Seems very inefficient, since the same transients can be processed many times. The token request goes faster, but it seems like the field based works "fast enough".

Q2: How long does it take to process one field? Started roughly 15.30. Took ~15 min for the token to yield alerts. Ended up with ~150 chunks. This took ~20 min to process and led to 80 saved candidates. O(800) T2 tickets created. This processing started ~16.10. 

Conclusion: we will go through each field (as many as possible) and try to parse data and run t2s in parallel. 

In [ ]:
dbname = 'allIa'

In [ ]:
import sys, os
import requests

In [ ]:
from ampel.secret.AmpelVault import AmpelVault
from ampel.secret.DictSecretProvider import DictSecretProvider
from ampel.dev.DevAmpelContext import DevAmpelContext

In [ ]:
# Create a secret vault
secrets = { 'slack/ztf_general/jno': os.environ["SNIaSLACK"],
                              'mongo/logger':{},
                              'mongo/writer':{},
                              'extcats/reader':{},
            'ztf/archive/auth' : {'user' : '', 'password' : ''},
            'datalab/pwd':os.environ["DATALABPWD"],
            "desycloud" : {},
            "ztf/archive/token": os.environ["ARCHIVE_TOKEN"],
          }
vault = AmpelVault([DictSecretProvider(secrets)])

In [ ]:
AMPEL_CONF = '/home/jnordin/github/ampel82/ampel_conf_630434.yaml'

In [ ]:
ctx = DevAmpelContext.load(
    config = AMPEL_CONF,
    db_prefix = dbname,
    purge_db = False,
    vault = vault
)

In [ ]:
t2w = ctx.new_context_unit(
    unit = 'T2Worker',
    process_name = 'T2Processor_7',
    log_profile = os.environ.get('log_profile', 'default')
)

In [ ]:
t2w.run()